In [1]:
import pandas as pd

# Exteract


In [2]:
# Load customer dim 
customer_dim = pd.read_excel("customer_dim.xlsx")

# Load seller dim 
seller_dim  = pd.read_csv("seller_dim.csv")

# Load product dim 
product_dim  = pd.read_csv("product_dim.csv")

# Load payment dim 
payment_dim  = pd.read_csv("payment_dim.csv")

# Load order dim 
order_dim  = pd.read_csv("order_dim.csv")

# Load order fact
order_fact = pd.read_csv("order_fact.csv")

# Load feedback dim 
feedback_dim  = pd.read_csv("feedback_dim.csv")


In [3]:
# preview data
print(customer_dim.head())
print('*---------------------------------------------------*')
print(seller_dim.head())
print('*---------------------------------------------------*')
print(product_dim.head())
print('*---------------------------------------------------*')
print(payment_dim.head())
print('*---------------------------------------------------*')
print(order_dim.head())
print('*---------------------------------------------------*')
print(order_fact.head())
print('*---------------------------------------------------*')
print(feedback_dim.head())


                      customer_name  customer_zip_code  \
0  861eff4711a542e4b93843c6dd7febb0              14409   
1  290c77bc529b7ac935b93aa66c333dc3               9790   
2  060e732b5b29e8181a18229c7b0b2b5e               1151   
3  259dac757896d24d7702b9acbbff3f3c               8775   
4  345ecd01c38d18a9036ed96c73b8d066              13056   

             customer_city customer_state  
0     KABUPATEN PEKALONGAN    JAWA TENGAH  
1              KOTA BEKASI     JAWA BARAT  
2           KOTA TANGERANG         BANTEN  
3  KABUPATEN BANDUNG BARAT     JAWA BARAT  
4       KOTA JAKARTA TIMUR    DKI JAKARTA  
*---------------------------------------------------*
                          seller_id  seller_zip_code          seller_city  \
0  3442f8959a84dea7ee197c632cb2df15            13023   KOTA JAKARTA TIMUR   
1  d1b65fc7debc3361ea86b5f14c68d2e2            13844  KOTA PADANG PANJANG   
2  ce3ad9de960102d0677a81f5d0bb7b2d            20031   KOTA JAKARTA BARAT   
3  c0f3eea2e14555b6faeea3

In [4]:
#check for null

print(customer_dim.isnull().sum())
print('*---------------------------------------------------*')
print(seller_dim.isnull().sum())
print('*---------------------------------------------------*')
print(product_dim.isnull().sum())
print('*---------------------------------------------------*')
print(payment_dim.isnull().sum())
print('*---------------------------------------------------*')
print(order_dim.isnull().sum())
print('*---------------------------------------------------*')
print(order_fact.isnull().sum())
print('*---------------------------------------------------*')
print(feedback_dim.isnull().sum())


customer_name        0
customer_zip_code    0
customer_city        0
customer_state       0
dtype: int64
*---------------------------------------------------*
seller_id          0
seller_zip_code    0
seller_city        0
seller_state       0
dtype: int64
*---------------------------------------------------*
product_id                      0
product_category              623
product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64
*---------------------------------------------------*
order_id                0
payment_sequential      0
payment_type            0
payment_installments    0
payment_value           0
dtype: int64
*---------------------------------------------------*
order_id                      0
customer_name                 0
order_status                  0
order_date              

the missing values have a meaning so leave it as it

# Transformation

In [5]:


# Drop duplicates, keeping only the last occurrence for each customer name 
customer_dim = customer_dim.drop_duplicates(subset='customer_name', keep='last')


In [6]:
# Dropping duplicates in each dataset
customer_dim.drop_duplicates(inplace=True)
seller_dim.drop_duplicates(inplace=True)
product_dim.drop_duplicates(inplace=True)
payment_dim.drop_duplicates(inplace=True)
order_dim.drop_duplicates(inplace=True)
order_fact.drop_duplicates(inplace=True)
feedback_dim.drop_duplicates(inplace=True)

In [7]:
# Converting date columns to datetime (example: if any date columns exist)

# order dim
order_dim['order_date'] = pd.to_datetime(order_dim['order_date'], errors='coerce')
order_dim['order_approved_date'] = pd.to_datetime(order_dim['order_approved_date'], errors='coerce')
order_dim['pickup_date'] = pd.to_datetime(order_dim['pickup_date'], errors='coerce')
order_dim['delivered_date'] = pd.to_datetime(order_dim['delivered_date'], errors='coerce')
order_dim['estimated_time_delivery'] = pd.to_datetime(order_dim['estimated_time_delivery'], errors='coerce')


# feedback dim
feedback_dim['feedback_form_sent_date'] = pd.to_datetime(feedback_dim['feedback_form_sent_date'], errors='coerce')
feedback_dim['feedback_answer_date'] = pd.to_datetime(feedback_dim['feedback_answer_date'], errors='coerce')






In [8]:
# Feature engineering: Calculate delivery time
order_fact['delivery_time'] = (order_dim['delivered_date'] - order_dim['order_date']).dt.days

In [9]:
# Add a surrogate key to customer_dim
#customer_dim['customer_id_sk'] = customer_dim['customer_name'] + "_" + customer_dim['customer_zip_code'].astype(str)
customer_dim['customer_id_sk'] = range(1, len(customer_dim) + 1)


In [10]:
# Create a mapping dictionary for surrogate keys
key_mapping = customer_dim.set_index('customer_name')['customer_id_sk'].to_dict()

# Map surrogate key to order_dim using user_name
order_dim['customer_id_sk'] = order_dim['customer_name'].map(key_mapping)


In [11]:
order_dim.columns

Index(['order_id', 'customer_name', 'order_status', 'order_date',
       'order_approved_date', 'pickup_date', 'delivered_date',
       'estimated_time_delivery', 'customer_id_sk'],
      dtype='object')

In [12]:
customer_dim.columns

Index(['customer_name', 'customer_zip_code', 'customer_city', 'customer_state',
       'customer_id_sk'],
      dtype='object')

In [13]:
# merging data 
#Merge order_dim with order_fact to include customer_surrogate_key
order_fact = pd.merge(order_fact, order_dim[['order_id', 'customer_id_sk']], on='order_id', how='left')

# Merge order_fact with other dimensions
merged_df = pd.merge(order_fact, seller_dim, on='seller_id', how='left')
merged_df = pd.merge(merged_df, product_dim, on='product_id', how='left')
merged_df = pd.merge(merged_df, payment_dim, on='order_id', how='left')
merged_df = pd.merge(merged_df, feedback_dim, on='order_id', how='left')



# Final merge with customer_dim to add customer-specific details
merged_df = pd.merge(merged_df, customer_dim, on='customer_id_sk', how='left')





In [14]:
merged_df.columns

Index(['order_id', 'order_item_id', 'product_id', 'seller_id',
       'pickup_limit_date', 'price', 'shipping_cost', 'delivery_time',
       'customer_id_sk', 'seller_zip_code', 'seller_city', 'seller_state',
       'product_category', 'product_name_lenght', 'product_description_lenght',
       'product_photos_qty', 'product_weight_g', 'product_length_cm',
       'product_height_cm', 'product_width_cm', 'payment_sequential',
       'payment_type', 'payment_installments', 'payment_value', 'feedback_id',
       'feedback_score', 'feedback_form_sent_date', 'feedback_answer_date',
       'customer_name', 'customer_zip_code', 'customer_city',
       'customer_state'],
      dtype='object')

In [15]:
#customer_dim.columns
#payment_dim.columns
#feedback_dim.columns
#order_dim.columns
#product_dim.columns
#seller_dim.columns
#customer_dim.columns

In [16]:
cols = order_fact.columns
cols

Index(['order_id', 'order_item_id', 'product_id', 'seller_id',
       'pickup_limit_date', 'price', 'shipping_cost', 'delivery_time',
       'customer_id_sk'],
      dtype='object')

In [17]:
final_fact = merged_df[cols]

In [18]:
final_fact.head()

,order_id,order_item_id,product_id,seller_id,pickup_limit_date,price,shipping_cost,delivery_time,customer_id_sk
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58900.0,13290.0,8.0,62649
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239900.0,19930.0,13.0,48397
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199000.0,17870.0,9.0,33086
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12990.0,12790.0,13.0,49256
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199900.0,18140.0,2.0,7129


In [19]:
final_fact.shape

(118318, 9)

In [20]:
merged_df[cols].shape

(118318, 9)

# Load

In [21]:

import pyodbc


server = 'DESKTOP-020PMUU\\SQLEXPRESS'  
database = 'ecommerce_dwh'

# Connection string for Windows Authentication
connection_string = f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;"

# Establish the connection
try:
    conn = pyodbc.connect(connection_string)
    cursor = conn.cursor()
    print("Connection successful!")

    # Commit the transaction
    conn.commit()
except Exception as e:
    print("Error:", e)
finally:
    # Close the connection
    conn.close()


Connection successful!


In [22]:
pip install pandas sqlalchemy pyodbc


Note: you may need to restart the kernel to use updated packages.


In [23]:

# from sqlalchemy import create_engine

# # Replace these with your actual database details
# server = 'DESKTOP-020PMUU\\SQLEXPRESS'  
# database = 'ecommerce_dwh'
# # Create the SQLAlchemy engine using Windows Authentication
# engine = create_engine(f'mssql+pyodbc://@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server;Trusted_Connection=yes')


In [24]:
from sqlalchemy import create_engine
from sqlalchemy.sql import text
from urllib.parse import quote_plus

# Define your ODBC connection string
odbc_connection_string = 'Driver={ODBC Driver 17 for SQL Server};' \
                          'Server=DESKTOP-020PMUU\\SQLEXPRESS;' \
                          'Database=ecommerce_dwh;' \
                          'Trusted_Connection=yes;'

# Encode the connection string to handle special characters
encoded_connection_string = quote_plus(odbc_connection_string)

# Create SQLAlchemy engine
engine = create_engine(f'mssql+pyodbc:///?odbc_connect={encoded_connection_string}')

# Test the connection
with engine.connect() as connection:
    result = connection.execute(text('SELECT 1'))
    print(result.fetchone())


(1,)


#### 1) load fact table
   

In [25]:
#customer_dim.columns
#payment_dim.columns
#feedback_dim.columns
#order_dim.columns
#product_dim.columns
#seller_dim.columns
#customer_dim.columns

In [26]:
final_fact.to_sql('order_fact', con=engine, if_exists='replace', index=False)


print("order_fact inserted successfully!")

order_fact inserted successfully!


##### 2)load dimensions

In [27]:
seller_dim.to_sql('seller_dim', con=engine, if_exists='replace', index=False)
print("seller_dim inserted successfully!")

seller_dim inserted successfully!


In [28]:
payment_dim.to_sql('payment_dim', con=engine, if_exists='replace', index=False)
print("payment_dim inserted successfully!")

payment_dim inserted successfully!


In [29]:
feedback_dim.to_sql('feedback_dim', con=engine, if_exists='replace', index=False)
print("feedback_dim inserted successfully!")

feedback_dim inserted successfully!


In [30]:
order_dim.to_sql('order_dim', con=engine, if_exists='replace', index=False)
print("order_dim inserted successfully!")

order_dim inserted successfully!


In [31]:
product_dim.to_sql('product_dim', con=engine, if_exists='replace', index=False)
print("product_dim inserted successfully!")


product_dim inserted successfully!


In [32]:
customer_dim.to_sql('customer_dim', con=engine, if_exists='replace', index=False)
print("customer_dim inserted successfully!")

customer_dim inserted successfully!
